In [3]:
#import libraries
import cmath
import numpy as np
import pandas as pd
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.environ import *

#load data
xls = pd.ExcelFile('data\case6.xlsx')
branch = pd.read_excel(xls, 'branch')
gen = pd.read_excel(xls, 'gen')
bus = pd.read_excel(xls, 'bus')
cost = pd.read_excel(xls, 'cost')

#Y matrix
base = 100
bus_len = bus.shape[0]
gen_len = len(gen['bus'].ravel())

Ybus = np.matrix(np.zeros([bus_len,bus_len]),dtype=complex)
shunt = np.zeros(bus_len, dtype=complex)
x = [pd.DataFrame() for i in range(bus_len)]
for i in range(bus_len):  
    x[i] = branch.loc[branch['fbus'] == i]
    fro = [j for j in x[i]['tbus']]
    fro = pd.DataFrame(fro)
    for j in range(len(fro)):
        Ybus[i-1,fro.loc[j][0]-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])
        Ybus[fro.loc[j][0]-1,i-1] =  -1/complex(pd.DataFrame(x[i]['r']).iloc[j][0],pd.DataFrame(x[i]['x']).iloc[j][0])
        shunt[j] = complex(0,sum(x[i]['b'])/2)

for i in range(bus_len):
    Ybus[i,i] = np.sum(Ybus[i,:])*(-1) + complex(0,sum(branch.loc[(branch['fbus']==i+1) | (branch['tbus']==i+1)]['b'])/2)


# declare variables
Pd = np.zeros(bus_len)
Qd = np.zeros(bus_len)
Vmax = np.zeros(bus_len)
Vmin = np.zeros(bus_len)
Pmax = np.zeros(bus_len)
Pmin = np.zeros(bus_len)
Qmax = np.zeros(bus_len)
Qmin = np.zeros(bus_len)
cost_lin = np.zeros(bus_len)
cost_quad = np.zeros(bus_len)

for i in range(bus_len):
    Pd[i] = bus['Pd'][i]/base
    Qd[i] = bus['Qd'][i]/base
    Vmax[i] = bus['Vmax'][i]
    Vmin[i] = bus['Vmin'][i]

for i in range(gen_len):
    k = i
    i = gen['bus'].ravel()[i]
    Pmax[i-1] = gen.loc[gen['bus'] == i]['Pmax'].ravel()[0]/base
    Pmin[i-1] = gen.loc[gen['bus'] == i]['Pmin'].ravel()[0]/base
    Qmax[i-1] = gen.loc[gen['bus'] == i]['Qmax'].ravel()[0]/base
    Qmin[i-1] = gen.loc[gen['bus'] == i]['Qmin'].ravel()[0]/base
    cost_lin[i-1] = cost['lin'][k]*base
    cost_quad[i-1] = cost['quad'][k]*base**2
      
#Create a model
model = pyo.ConcreteModel()

model.IDX1 = range(gen_len)
model.IDX2 = range(bus_len)
model.Pg = pyo.Var(model.IDX2)
model.Qg = pyo.Var(model.IDX2)
model.t = pyo.Var(model.IDX2)
model.v = pyo.Var(model.IDX2)
for i in model.IDX2:
    model.Pg[i].fix(0) 
    model.Qg[i].fix(0) 
    
for i in model.IDX1:
    model.Pg[gen['bus'][i]-1].fixed = False
    model.Qg[gen['bus'][i]-1].fixed = False


# declare constraints
model.c = pyo.ConstraintList()
#model.c.add(expr = sum(model.Pg[i] for i in model.IDX2) == sum(Pd))
for i in range(bus_len):
    model.c.add(expr = model.Pg[i] <= Pmax[i])
    model.c.add(expr = model.Pg[i] >= Pmin[i])
    model.c.add(expr = model.v[i] <= Vmax[i])
    model.c.add(expr = model.v[i] >= Vmin[i])    

#Bus voltage angle initial condition
model.c.add(model.t[0] == 0)

#Nodal equations
for i in range(bus_len):
    model.c.add(expr = sum([model.v[i]*model.v[j]*cmath.polar(Ybus[i,j])[0]*cos(model.t[i]-model.t[j]-cmath.polar(Ybus[i,j])[1]) for j in range(bus_len)]) - model.Pg[i] + Pd[i]== 0)
    model.c.add(expr = sum([model.v[i]*model.v[j]*cmath.polar(Ybus[i,j])[0]*sin(model.t[i]-model.t[j]-cmath.polar(Ybus[i,j])[1]) for j in range(bus_len)]) - model.Qg[i] + Qd[i]== 0)

#Line flow constraints
for i in range(len(branch)): 
    x = int(branch.loc[i]['fbus'])-1
    y = int(branch.loc[i]['tbus'])-1
    model.c.add(expr = abs(((model.v[x]**2)*cmath.polar(Ybus[x,y])[0])*cos(cmath.polar(Ybus[x,y])[1]) - model.v[x]*model.v[y]*cmath.polar(Ybus[x,y])[0]*cos(model.t[x]-model.t[y]-cmath.polar(Ybus[x,y])[1])) <= int(branch.loc[i]['rateA'])/base)
      
# declare objective
model.cost = pyo.Objective( 
    expr = sum(cost['cons']) + 
    (sum(model.Pg[i]*cost_lin[i] for i in model.IDX2)) + 
    (sum(cost_quad[i]*model.Pg[i]**2 for i in model.IDX2))
) 

# solve
SolverFactory('ipopt').solve(model).write()


#Print results
print('Cost = ',model.cost())



# #Print results
# print('Cost = ',model.cost())
# print('Generator 1 P= ',model.p1())
# print('Generator 2 P= ', model.p2())
# print('Generator 3 P= ', model.p3())
# print('Generator 1 Q= ',model.q1())
# print('Generator 2 Q= ', model.q2())
# print('Generator 3 Q= ', model.q3())
# print('Bus angle N1 = ',model.v1())
# print('Bus angle N2 = ',model.v2())
# print('Bus angle N3 = ',model.v3())
# print('Bus angle N4 = ',model.v4())
# print('Bus angle N5 = ',model.v5())
# print('Bus angle N6 = ',model.v6())


# Debug code
# from pyomo.util.infeasible import log_infeasible_constraints
# SolverFactory('gurobi').solve(model)
# log_infeasible_constraints(model)



# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 48
  Number of variables: 18
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.11.1\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.04986453056335449
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0
Cost =

In [153]:
for i in range(bus_len):
    print(model.v[i].value)

1.0700000104663556
1.070000010341769
1.0667002542288035
1.0381472653695931
1.036279259510103
1.0426348602279925
